In [ ]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)


from IPython.display import display
import pandas as pd

import msticpy.sectools as sectools
import msticpy.nbtools as mas
from msticpy.sectools import VTLookup, IoCExtract
from msticpy.sectools.geoip import GeoLiteLookup, IPStackLookup
from msticpy.nbtools.entityschema import IpAddress, GeoLocation
from msticpy.nbtools.foliummap import FoliumMap

In [ ]:
# Enter your VT Key here
vt_key = mas.nbwidgets.GetEnvironmentKey(env_var='VT_API_KEY',
                           help_str='To obtain an API key sign up here https://www.virustotal.com/',
                           prompt='Virus Total API key:')
vt_key.display()

In [ ]:
# Create an instance of the class

vt_lookup = sectools.VTLookup(vt_key.value, verbosity=2)

In [ ]:
# How these map to VT lookup types
VTLookup._VT_TYPE_MAP

In [ ]:
# Load a CSV to build the iplist DF, aka the public IP list to enrich  
# ATTENTION: set the first line of the file to 'IP\r\n'
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)

iplist = pd.read_csv(filename)
print(iplist)

In [ ]:
# Check the reputation of the iplist in VirusTotal
ioc_extractor = IoCExtract()
vt_lookup = VTLookup(vt_key.value, verbosity=2)
output_df = ioc_extractor.extract(data=iplist, 
                                  columns=['IP'], 
                                  ioc_types=vt_lookup.supported_ioc_types)
output_df

In [ ]:
from past.builtins import xrange
vt_lookup = VTLookup(vt_key.value, verbosity=2)
vt_res = [[]]

# Submit a subset of the found IoCs (ignore dns since a lot of )
vt_results = vt_lookup.lookup_iocs(data=output_df, 
                                   type_col='IoCType', 
                                   src_col='Observable')

#clean the DF, deleting uninteresting cols
vt_res = vt_results.drop(columns=['IoCType', 'Status', 'ResponseCode', 'RawResponse', 'Resource', 'SourceIndex', 'VerboseMsg', 'Resource', 'ScanId', 'Permalink', 'MD5', 'SHA1', 'SHA256', 'ResolvedIPs'])


In [ ]:
iplocation = GeoLiteLookup()
vt_res = iplocation.df_lookup_ip(vt_res,'Observable')
display(vt_res)

In [ ]:
# Load a CSV to build the iplist DF, aka the public IP list to enrich  
from tkinter import Tk
from tkinter.filedialog import asksaveasfile

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = asksaveasfile() # show an "Open" dialog box and return the path to the selected file

vt_res.to_csv(filename, index=False) 

In [ ]:
folium_map = FoliumMap(location=[vt_res.Latitude.mean(), vt_res.Longitude.mean()], zoom_start=1)
# Set Icon properties to display
icon_props = {"color": "green"}

def create_ip_entity(row):
    ip_ent = IpAddress(Address=row["Observable"])
    geo_loc = create_geo_entity(row)
    ip_ent.Location = geo_loc
    return ip_ent

def create_geo_entity(row):
    # get subset of fields for GeoLocation
    loc_props = row[["CountryCode", "CountryName","State", "City", "Longitude", "Latitude"]]
    geo_loc = GeoLocation(**loc_props.to_dict())
    return geo_loc

# Set additional properties to display in tooltips
# in AdditionalData property of IpAddress entities
geo_locs = list(vt_res.apply(create_geo_entity, axis=1).values)
ip_ents = list(vt_res.apply(create_ip_entity, axis=1).values)
folium_map.add_ip_cluster(ip_entities=ip_ents, **icon_props)

# display the map (or just have folium_map as the last statement
# of the cell - Jupyter will use display() automatically)
display(folium_map)
#folium_map.save('result.html')
#os.system('explorer.exe result.html')